In [1]:
# ===== COPY FROM https://github.com/OGGM/oggm-edu-notebooks/blob/master/oggm-tuto/dem_comparison.ipynb =====
# The RGI Id of the glaciers you want to look for
# Use the original shapefiles or the GLIMS viewer to check for the ID: https://www.glims.org/maps/glims
# rgi_id = 'RGI60-11.00897'

# The default is to test for all sources available for this glacier
# Set to a list of source names to override this
sources = None
# Where to write the plots. Default is in the current working directory
plot_dir = ''
# The RGI version to use
# V62 is an unofficial modification of V6 with only minor, backwards compatible modifications
prepro_rgi_version = 62
# Size of the map around the glacier. Currently only 10 is available
prepro_border = 10
# Degree of processing level.  Currently only 1 is available.
from_prepro_level = 1

# The sources can be given as parameters
if sources is not None and isinstance(sources, str):
    sources = sources.split(',')

# # Plotting directory as well
# if not plot_dir:
#     plot_dir = './' + rgi_id
# import os
# plot_dir = os.path.abspath(plot_dir)

In [2]:
import pandas as pd
import numpy as np
from oggm import cfg, utils, workflow, tasks, graphics, GlacierDirectory
import xarray as xr
import os
import geopandas as gpd
import salem
import matplotlib.pyplot as plt
import shutil

from oggm.utils import DEM_SOURCES
from oggm.workflow import init_glacier_regions

import globaldebris_input as input

In [3]:
# Use OGGM to download the data
# cfg.initialize(logging_level='CRITICAL')
cfg.initialize(logging_level='WORKFLOW')
cfg.PATHS['working_dir'] = '/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/oggm_dems/'
# cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-DEMS', reset=True)

# OGGM params
cfg.PARAMS['use_intersects'] = False
cfg.PARAMS['flowline_dx'] = 1
cfg.PARAMS['use_multiprocessing'] = False
cfg.PARAMS['use_multiple_flowlines'] = False
cfg.PARAMS['border'] = prepro_border

2020-02-24 19:03:28: oggm.cfg: Using configuration file: /Users/davidrounce/anaconda3/envs/pygem_v3_viapip/lib/python3.5/site-packages/oggm/params.cfg
2020-02-24 19:03:28: oggm.cfg: Multiprocessing switched ON according to the parameter file.
2020-02-24 19:03:28: oggm.cfg: Multiprocessing: using all available processors (N=4)


In [4]:
print('\n\nDOWNLOAD ALL THE DEMS FOR A GIVEN REGION\n\n')

download_dems = True
rgi_region = '03'

if download_dems:
    # URL of the preprocessed GDirs
    gdir_url = 'https://cluster.klima.uni-bremen.de/data/gdirs/dems_v0/'

    # Hack to download all glaciers at once
    shp = gpd.read_file(utils.get_rgi_region_file(rgi_region))

    # Hack to download only a subset
    # shp = shp.loc[shp.RGIId == 'RGI60-11.00897'] 

    # We use OGGM to download the data
    # gdir = init_glacier_regions([rgiid], from_prepro_level=1, prepro_border=10, 
    #                             prepro_rgi_version='62', prepro_base_url=gdir_url)
    gdirs = init_glacier_regions(shp, from_prepro_level=1, prepro_border=10, 
                                 prepro_rgi_version='62', prepro_base_url=gdir_url)
    
    



DOWNLOAD ALL THE DEMS FOR A GIVEN REGION




2020-02-24 19:03:33: oggm.workflow: init_glacier_regions from prepro level 1 on 4556 glaciers.


KeyboardInterrupt: 

In [10]:
select_bestdem = True
if select_bestdem:
    import shutil

    main_glac_rgi = input.selectglaciersrgitable(rgi_regionsO1=input.roi_rgidict[input.roi],
                                                 rgi_regionsO2='all', rgi_glac_number='all')
    for rgiid in main_glac_rgi.RGIId.values:
    # for rgiid in [main_glac_rgi.RGIId.values[0]]:
#         print(rgiid)
        dem_fp = input.oggm_fp + 'per_glacier/' + rgiid[0:8] + '/' + rgiid[0:11] + '/' + rgiid + '/SRTM/'
        output_dem_fp = input.oggm_fp + 'dem_qc/' + rgiid[0:8] + '/'
        if os.path.exists(output_dem_fp) == False:
            os.makedirs(output_dem_fp)
        output_dem_fn = rgiid + '-dem.tif'
        shutil.copy(dem_fp + 'dem.tif', output_dem_fp + output_dem_fn)

All glaciers within region(s) 18 are included in this model run.
This study is focusing on 3537 glaciers in region [18]


In [7]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(input.debriscover_fp + input.debriscover_fn_dict[input.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > input.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > input.dc_area_threshold))
                        & (dc_shp['Area'] > input.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)
# print(dc_shp_subset.RGIId.values)
dc_shp_subset

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Name,DC_Area,DC_BgnDate,DC_EndDate,DC_CTSmean,DC_Area_%,area_singl,DC_Area_v2,DC_Area__1,geometry
0,RGI60-18.00171,G168039E44594S,19789999,-9999999,168.037603,-44.591427,18,1,2.790,1160,...,None,810000,2013,2017,20.105890,29.032,785599,785599,28.158,POLYGON ((168.0324222616754 -44.59845631393821...
1,RGI60-18.00179,G168020E44606S,19789999,-9999999,168.018668,-44.605576,18,1,2.189,850,...,None,240300,2013,2017,24.129812,10.978,151562,151562,6.924,POLYGON ((168.0065651847922 -44.61614576214832...
2,RGI60-18.00569,G168242E44591S,19789999,-9999999,168.235877,-44.590591,18,1,2.559,1503,...,None,373500,2013,2017,10.591865,14.596,133148,295086,11.531,(POLYGON ((168.2252099846863 -44.5903434235009...
3,RGI60-18.00631,G168624E44538S,19789999,-9999999,168.620338,-44.539415,18,1,2.472,1465,...,None,445500,2013,2017,11.149417,18.022,21598,343770,13.907,(POLYGON ((168.6111261007399 -44.5434378945606...
4,RGI60-18.00661,G168556E44463S,19789999,-9999999,168.553044,-44.465521,18,1,7.027,1073,...,None,1637100,2013,2017,11.708163,23.297,1614468,1614468,22.975,POLYGON ((168.5125621499583 -44.47246305362818...
5,RGI60-18.00686,G168609E44455S,19789999,-9999999,168.602431,-44.456298,18,1,9.656,1036,...,None,1511100,2013,2017,11.280399,15.649,1165166,1429743,14.807,(POLYGON ((168.6098306539947 -44.4550955394131...
6,RGI60-18.00707,G168578E44439S,19789999,-9999999,168.578927,-44.439472,18,1,5.191,1282,...,None,741600,2013,2017,9.443480,14.286,25197,691794,13.327,(POLYGON ((168.5645460346074 -44.4287509171137...
7,RGI60-18.00716,G168482E44469S,19789999,-9999999,168.481966,-44.469631,18,1,4.053,1292,...,None,583200,2013,2017,11.178898,14.389,519199,540795,13.343,(POLYGON ((168.4777845708828 -44.4654881006597...
8,RGI60-18.00718,G168495E44485S,19789999,-9999999,168.487470,-44.487016,18,1,2.311,1136,...,None,387000,2013,2017,9.974349,16.746,158259,322029,13.935,(POLYGON ((168.4798509735454 -44.4963225657939...
9,RGI60-18.00815,G168358E44502S,19789999,-9999999,168.355187,-44.499472,18,1,2.833,1186,...,None,749700,2013,2017,10.124881,26.463,683812,683812,24.137,POLYGON ((168.3594192693275 -44.50602347041639...


In [8]:
process_flowlines = True

shp_all = None
if process_flowlines:
    # Hack to download all glaciers at once
    for rgi_region in input.roi_rgidict[input.roi]:
        shp = gpd.read_file(utils.get_rgi_region_file(str(rgi_region)))
        
        if shp_all is None:
            shp_all = shp
        else:
            shp_all = pd.concat([shp_all, shp], axis=0)

    shp_all.reset_index(inplace=True, drop=True)
    
    dc_list = list(dc_shp_subset.RGIId.values)
    shp_list = list(shp_all.RGIId.values)
    shp_idx = np.array([shp_list.index(x) for x in dc_list])
    shp = shp_all.loc[shp_idx,:]

    # This SHOULD not do anything, just checking that the directories are there - but it does somthing as of today
    # after https://github.com/OGGM/oggm/pull/957 is merged, it should work
    gdirs = init_glacier_regions(shp)

    for gdir in gdirs:
#     for gdir in [gdirs[0]]:

#         print((gdir.rgi_area_m2, gdir.rgi_id, gdir.rgi_date, gdir.is_icecap))

    #     # Plot on Google Earth
    #     from oggm import graphics
    #     graphics.plot_googlemap(gdir)

        # DEMs
        if sources is None:
            sources = [src for src in os.listdir(gdir.dir) if src in utils.DEM_SOURCES]

        print('RGI ID:', gdir.rgi_id)
    #     print('Available DEM sources:', sources)
    #     print('Glacier directory:', gdir.dir)

        # ===== Pick DEM =====
        # Pick one source now (see example below to read all of them)
        src = 'SRTM'
        demfile = os.path.join(gdir.dir, src, 'dem.tif')
        srcfile = os.path.join(gdir.dir, src, 'dem_source.txt')

        # We copy the selected DEM at the expected location in the directory 
        # so that OGGM can work on it later
        shutil.copyfile(demfile, os.path.join(gdir.dir, 'dem.tif'))
        shutil.copyfile(srcfile, os.path.join(gdir.dir, 'dem_source.txt'))

        # Just to plot things, we also read it in a xr Dataset
        ods = xr.Dataset()
        with xr.open_rasterio(demfile) as ds:
            data = ds.sel(band=1).load() * 1.
            ods['DEM'] = data.where(data > -100, np.NaN)

        with xr.open_rasterio(gdir.get_filepath('glacier_mask')) as ds:
            ods['mask'] = ds.sel(band=1).load()

        # # We could also read all of them
        # ods = xr.Dataset()
        # for src in sources:
        #     demfile = os.path.join(gdir.dir, src) + '/dem.tif'
        #     print(demfile)
        #     with xr.open_rasterio(demfile) as ds:
        #         data = ds.sel(band=1).load() * 1.
        #         ods[src] = data.where(data > -100, np.NaN)
        #     sy, sx = np.gradient(ods[src], gdir.grid.dx, gdir.grid.dx)
        #     ods[src + '_slope'] = ('y', 'x'),  np.arctan(np.sqrt(sy**2 + sx**2))
        # with xr.open_rasterio(gdir.get_filepath('glacier_mask')) as ds:
        #     ods['mask'] = ds.sel(band=1).load()


    #     ods['mask'].plot();
    #     plt.figure();
    #     ods['DEM'].plot();


        # ===== COMPUTE FLOWLINE WITH OGGM =====
        task_list = [
            tasks.glacier_masks,
            tasks.compute_centerlines,
            tasks.initialize_flowlines,
            tasks.catchment_area,
            tasks.catchment_intersections,
            tasks.catchment_width_geom,
            tasks.catchment_width_correction,
        ]

        try:
            for task in task_list:
                task(gdir)

        #     graphics.plot_catchment_width(gdir, corrected=True)

            fls = gdir.read_pickle('inversion_flowlines')
            assert len(fls) == 1
            fl = fls[0]

            # ----- WIDTHS -----
            widths = fl.widths_m
            h = fl.surface_h

            df = pd.DataFrame(np.concatenate((h[:,np.newaxis], widths[:,np.newaxis]), axis=1), columns=['elev', 'width_m'])
            widths_fp = gdir.dir + '/../../../../widths/'
            if os.path.exists(widths_fp) == False:
                os.makedirs(widths_fp)
            widths_fn = gdir.rgi_id.split('-')[1] + '-widths_m.csv'
            df.to_csv(widths_fp + widths_fn, index=False)

        #     x = fl.dis_on_line * fl.dx_meter
        #     plt.plot(x, widths)
        #     plt.xlabel('Distance along flowline')
        #     plt.ylabel('Widths')

        #     plt.plot(x, h);
        #     plt.xlabel('Distance along flowline');
        #     plt.ylabel('Altitude');


            # average the with of the lowest 25%
        #     print(np.mean(widths[h < np.quantile(h, 0.25)]))

        #     # average the width of the 10 to 40%
        #     np.mean(widths[(h < np.quantile(h, 0.40)) & (h > np.quantile(h, 0.1))])
        except:
            print('PASSED!')
            pass


RGI ID: RGI60-18.00171
RGI ID: RGI60-18.00179
RGI ID: RGI60-18.00569
RGI ID: RGI60-18.00631
RGI ID: RGI60-18.00661
RGI ID: RGI60-18.00686
RGI ID: RGI60-18.00707
RGI ID: RGI60-18.00716
RGI ID: RGI60-18.00718
RGI ID: RGI60-18.00815
RGI ID: RGI60-18.00853
RGI ID: RGI60-18.00891
RGI ID: RGI60-18.00947
RGI ID: RGI60-18.00996
RGI ID: RGI60-18.01018
RGI ID: RGI60-18.01130
RGI ID: RGI60-18.01307
RGI ID: RGI60-18.01852
RGI ID: RGI60-18.01889
RGI ID: RGI60-18.01958
RGI ID: RGI60-18.01959
RGI ID: RGI60-18.02210
RGI ID: RGI60-18.02230
RGI ID: RGI60-18.02257
RGI ID: RGI60-18.02270
RGI ID: RGI60-18.02276
RGI ID: RGI60-18.02298
RGI ID: RGI60-18.02342
RGI ID: RGI60-18.02436
RGI ID: RGI60-18.02450
RGI ID: RGI60-18.02472
RGI ID: RGI60-18.02499
RGI ID: RGI60-18.02502
RGI ID: RGI60-18.02504
RGI ID: RGI60-18.02505
RGI ID: RGI60-18.02508
RGI ID: RGI60-18.03066
RGI ID: RGI60-18.03078
RGI ID: RGI60-18.03156
RGI ID: RGI60-18.03167
RGI ID: RGI60-18.03181
RGI ID: RGI60-18.03191
RGI ID: RGI60-18.03209


In [10]:
print('\n\nDONE!\n\n')



DONE!


